## Group 6:
### AU1741001: Akash Tike
### AU1741011: Smit Mandavia
### AU1741068: Parth Maniyar
### AU1741095: Shaunak Vyas

## Collaborative filtering (Streaming Distributed stochastic gradient descent for matrix factorization)

In [0]:
from random import randint 
import random
import multiprocessing
import time
import numpy as np

Importing libaries

In [0]:
class U:
  def __init__(self, i , j ):
    self.matrix = np.random.uniform( low=0,high=1.3,size=(i, j))

  def __str__(self):  
    return 'marix U is' + self.matrix.tostring()

class V:
  def __init__(self, i , j ):
    self.matrix = np.random.uniform( low=0,high=1.3,size=(i, j))
  def __str__(self):  
    return 'marix V is' +self.matrix.tostring()

*   The matrix U and matrix V are intialized with random values in range $(0,1.3)$.

*   The range is $0$ to $1.3$ because $k=6$ and the dot product will be between 0 to $1.3*1.3*6$ ~= $10$.     
If we seed the matrix $U$ and $V$ properly they it will converge faster

*   the `__str__` function is implemented for printing the class object

In [0]:
class Rating: 
  def __init__(self, i , j , rating , block_index_i, block_index_j):
    self.i = i
    self.j = j
    self.rating = rating
    self.block_index_i = block_index_i
    self.block_index_j = block_index_j
  
  def __str__(self):  
    return "From str method of Rating:i  is % d, j is % d, blk_i is % d, blk_j is % d , rating is % d" % (self.i, self.j,self.block_index_i,self.block_index_j,self.rating)  


*   Class rating will be used to store rating updates.   
*   `i,j` denotes the user index and item index respectively.       
*   `block_index_i` and `block_index_i` denotes the index of block in which they lie after partitioning the matrix into $d*d$ smaller matrices.

In [0]:
class Turn:
  def __init__(self, p , q ):
    self.p = p
    self.q = q

  def __str__(self):  
    return "From str method of Turn: p is % d, q is % d" % (self.p, self.q)  

*   Class Turn will be used give and close turn of a worker and it will also contain information about which block is to be updated.   
*   `p` and `q` denotes the index of block in which needs to be updated.

In [0]:
class Rating_saved:
  def __init__(self, rating ):
    self.i = rating.i
    self.j = rating.j
    self.rating = rating.rating
    self.block_index_i = rating.block_index_i
    self.block_index_j = rating.block_index_j
  
  def __str__(self):  
    return "From str method of Rating:i  is % d, j is % d, blk_i is % d, blk_j is % d , rating is % d" % (self.i, self.j,self.block_index_i,self.block_index_j,self.rating)  

*   The acknowlegment for ratings saved from worker to master 
*   We will discuss later in code why is this important 

In [0]:
class Data: 
  def __init__(self, isP, index , data ):
    self.isP = isP
    self.index = index
    self.data = data

  def __str__(self):  
    return "From str method of Data: isP is % d, index is % d , data is " % (self.isP, self.index)  

*   Class Data will be used transfer data between worker and matrix store.        
*   `isP` is `True` if block is a U type block and it is `False` if it is a V type block.   

In [0]:
class Request: 
  def __init__(self, isP, index, my_index):
    self.isP = isP
    self.index = index
    self.my_index = my_index
  def __str__(self):  
    return "From str method of Request: isP is % d, index is % d , my_index is %d " % (self.isP, self.index, self.my_index)   


*   Class Request will be used to request data from matrix store.        
*   `isP` is `True` if requested block is a U type block and it is `False` if it is a V type block.   
*   `index` denotes the index of the requested block.
*   `my_index` denotes the index of the worker who is requesting the data


In [0]:
def matrix_store(rating_matrix, is_blocked_U, is_blocked_V , matrix_U, matrix_V , matrix_store_request , matrix_store_data_send , matrix_store_data_update, command) :

  while True:
    while matrix_store_data_update.qsize()>0:    
      data = matrix_store_data_update.get()

      if data.isP:
        matrix_U[data.index].matrix = data.data
        is_blocked_U[data.index]=False
      else:
        matrix_V[data.index].matrix = data.data
        is_blocked_V[data.index]=False
      
    while command.qsize()>0:
      cmd = command.get()
      if cmd == 'Exit': 
        return
      if cmd == 'Print':
        prediction = np.dot(matrix_U[0].matrix.T,matrix_V[0].matrix)
        print(prediction)  
        print(rating_matrix[0][0])
        sum_of_error=0
        sum_of_square=0
        cnt = 0
        for i in range(len(prediction)):
            for j in  range(len(prediction[0])):
                if rating_matrix[0][0][i][j]!=0:
                    sum_of_error = sum_of_error+ abs( rating_matrix[0][0][i][j]-prediction[i][j]) 
                    sum_of_square = sum_of_square + abs( rating_matrix[0][0][i][j]-prediction[i][j])*abs( rating_matrix[0][0][i][j]-prediction[i][j]) 
                    cnt = cnt+1

        is_rated = (rating_matrix[0][0] != 0)*1
        print("MAE: ",np.sum(abs(rating_matrix[0][0]-prediction)*is_rated)/np.sum(is_rated))
        print("MSE: ",np.sum((np.square(rating_matrix[0][0] - prediction)*is_rated))/np.sum(is_rated),flush=True)
        print("MAE: ", sum_of_error/cnt )
        print("MSE: ", sum_of_square/cnt )
          

    if matrix_store_request.qsize()>0:
      req =  matrix_store_request.get()
      if req.isP:
          if is_blocked_U[req.index]:
             matrix_store_request.put(req)  
             continue
      else:
          if is_blocked_V[req.index]:
             matrix_store_request.put(req)  
             continue

      if req.isP:
        matrix_store_data_send[req.my_index].put(Data(True, req.index, matrix_U[req.index].matrix ))
        is_blocked_U[req.index]=True
      else:
        matrix_store_data_send[req.my_index].put(Data(False, req.index, matrix_V[req.index].matrix))
        is_blocked_V[req.index]=True
  

*   The first while loop updates the martrices received from worker after updating. It also unblocks the blocks for further updates. 
*   The second while loop is made for debugging purpose. We can give various commands to the store from main fucntion and control the flow of code. 
*   Third is if condition instead of loop. It is because the least important task is assigning matrix to new workers as comapared to other two tasks. Store cannot get overwhelmed by number of requests of data and can respond to other two tasks. 
*   Third part of code allots martrix as requested by workers and also protects similtaneous updates by blocking matrix allocated. It cannot cause a deadlock because main function also takes care of protecting data simultaneous update.     
*   This extra protection is just for the cases when data store is slow in updating received data from workers and main function thinks the worker already stopped working.       
*   It simply puts the request back to end of queue if a blocked matrix block is requested


In [0]:
def worker( worker_rating_saved , is_updated, rating_matrix , new_ratings, turn, turn_over, matrix_store_request , matrix_store_data_recieve , matrix_store_data_update , command):
  cnt = 0 
  my_index = 0
  while True: 
    while command.qsize()>0: # its use is same as command used in matrix store
      cmd = command.get()
      if cmd == 'Exit': 
        return
      if cmd == 'Print':
        matrix_store_request.put(Request(True,my_index,0))
        matrix_store_request.put(Request(False,my_index,0))
        data = matrix_store_data_recieve.get()
        data2 = matrix_store_data_recieve.get()
        
        if data.isP: 
          U_data = data.data
          V_data = data2.data 
        else :    
          U_data = data2.data
          V_data = data.data 

        is_rated = (rating_matrix[0] != 0)*1
        new_prediction = np.dot(U_data.T,V_data)

        print('Rating matrix : ', rating_matrix[0])
        print('Prediction : ', new_prediction)
      
        if np.sum(is_rated)>0 :
          print(my_index, ' ', 0 , " Print MAE: ",np.sum(abs(rating_matrix[0]-new_prediction)*is_rated)/np.sum(is_rated))
          print(my_index, ' ', 0 , " Print MSE: ",np.sum((np.square(rating_matrix[0] - new_prediction)*is_rated))/np.sum(is_rated),flush=True)
        else : 
          print(my_index, ' '  ,0 , " Empty",flush=True)
        
        matrix_store_data_update.put(data)
        matrix_store_data_update.put(data2)
      

    while new_ratings.qsize()>0: # If there are new ratings available  
      new_rating = new_ratings.get()
      rating_matrix[new_rating.block_index_j][new_rating.i][new_rating.j] = new_rating.rating # store them
      is_updated[new_rating.block_index_j][new_rating.i][new_rating.j] = 0   # and mark it to update when turn is received
      worker_rating_saved.put(Rating_saved(new_rating))    # Send acknowledgement to master that this rating is saved
      cnt = cnt+1 

    if turn.qsize()>0:
      my_turn  = turn.get()  # waiting for turn
      my_index =  my_turn.p
      print(my_turn.p, ' ', my_turn.q, ' : Number of points received after last update : ' , cnt)
      cnt = 0 
      
      # Requesting data from data store
    
      matrix_store_request.put(Request(True,my_turn.p,my_turn.p))
      matrix_store_request.put(Request(False,my_turn.q,my_turn.p))
      
      # Waiting for data (U and V) from data store
    
      data = matrix_store_data_recieve.get()
      data2 = matrix_store_data_recieve.get()
      

      if data.isP: 
        U_data = data.data
        V_data = data2.data 
      else :    
        U_data = data2.data
        V_data = data.data 

      # Caculating prediction before updating U and V

      is_rated = (rating_matrix[my_turn.q] != 0)*1
      new_prediction = np.dot(U_data.T,V_data)

      if np.sum(is_rated)>0 :
        print(my_turn.p, ' ', my_turn.q, " Before MAE: ",np.sum(abs(rating_matrix[my_turn.q]-new_prediction)*is_rated)/np.sum(is_rated))
        print(my_turn.p, ' ', my_turn.q, " Before MSE: ",np.sum((np.square(rating_matrix[my_turn.q] - new_prediction)*is_rated))/np.sum(is_rated),flush=True)
      else : 
        print(my_turn.p, ' '  ,my_turn.q, "Empty",flush=True)  # There are no ratings in given block. This can happen if worker is slow in updting rating received.
        turn_over.put(my_turn)
        matrix_store_data_update.put(data)
        matrix_store_data_update.put(data2)
        continue  # we will skip updating as there is nothing to update
      # the main function will definitely call this once more( even if there are no futher updates) if this is empty 
      # because the sends acknowlegment for ratings received 

    
      # Now updating matrix using normal algoithm on assigned block of martix as implemented and discussed in 5th assignment


      #learning rate
      eta = 0.0105

      #regularization parameter to control overfitting
      lamda = 15
      
      epoch=1
      
      for itr in range(epoch):
        for i in range(len(U_data[0])):
          for j in range(len(V_data[0])):
            if rating_matrix[my_turn.q][i][j]!=0 and is_updated[my_turn.q][i][j]==0:    
              new_Ui = U_data[:,i] - 2*eta*((np.dot(U_data[:,i],V_data[:,j])-rating_matrix[my_turn.q][i][j])*V_data[:,j] + (lamda/len(V_data[0]))*U_data[:,i])
              new_Vj = V_data[:,j] - 2*eta*((np.dot(U_data[:,i],V_data[:,j])-rating_matrix[my_turn.q][i][j])*U_data[:,i] + (lamda/len(U_data[0]))*V_data[:,j])
                                
              U_data[:,i] = new_Ui
              V_data[:,j] = new_Vj

      for i in range(len(U_data[0])):
          for j in range(len(V_data[0])):
             if rating_matrix[my_turn.q][i][j]!=0:
                is_updated[my_turn.q][i][j]=1

      # Completed updates successfully 

      # Calculating new predictions 
      new_prediction = np.dot(U_data.T,V_data)
      
      # Printing MAE and MSE after updates, also prints block index so that we can compare (with before update MAE and MSE) and identify accuracy of each block
      if np.sum(is_rated)>0 :
        print(my_turn.p, ' ', my_turn.q," After MAE: ",np.sum(abs(rating_matrix[my_turn.q]-new_prediction)*is_rated)/np.sum(is_rated))
        print(my_turn.p,' ', my_turn.q, " After MSE: ",np.sum((np.square(rating_matrix[my_turn.q] - new_prediction)*is_rated))/np.sum(is_rated),flush=True)
      else : # This should't happen at all ( /0 error)
        print(my_turn.p,' ', my_turn.q, "Error",flush=True)
      
      if data.isP: 
        data.data = U_data 
        data2.data = V_data   
      else :    
        data2.data = U_data 
        data.data = V_data  

      # Sending data back to marix store
      
      matrix_store_data_update.put(data)
      matrix_store_data_update.put(data2)

      # Complete turn by notifying master   
      turn_over.put(my_turn)


*   The second while loop is made for debugging purpose. We can give various commands to the store from main fucntion and control the flow of code.  
*   Currently `Print` command prints predicted ratings, actual ratings as well as mean absolute and mean square error of first block of current worker. 


In [0]:
def master( set_s , is_blocked_row , is_blocked_column, worker_rating_saved , book_keeping, new_ratings, worker_new_rating, worker_turn, worker_turn_over, d, command ) : 
  
  while True:
    while command.qsize()>0:  # its use is same as command used in matrix store
      cmd = command.get()
      if cmd == 'Exit': 
        return

    time_out = 50  # to avoid master from getting overwhelmed by stream
    while new_ratings.qsize()>0 and time_out>0:  # if new ratings are received then send it to worker
      new_rating = new_ratings.get()
      worker_new_rating[new_rating.block_index_i].put(new_rating)
      time_out = time_out-1
    
    time_out = 2  # wait for few moments if workers have not yet completed  their work 
    while worker_turn_over.qsize()>0 or len(set_s)==0:
      if time_out<0:
          break
      if worker_turn_over.qsize()==0:
          time_out = time_out-1
          time.sleep(0.20)
          continue
      turn = worker_turn_over.get()   # worker completed work
      is_blocked_row[turn.p]=False    # Unblock row and column of block
      is_blocked_column[turn.q]=False   

      # adding unblocked book keeing to set
      for i in range(len(book_keeping[turn.p])):  
        if book_keeping[turn.p][i]==True and is_blocked_column[i]==False:
          set_s.add((turn.p,i))

      for i in range(len(book_keeping)):
        if book_keeping[i][turn.q]==True and is_blocked_row[i]==False:
          set_s.add((i,turn.q))
    
    time_out = 1000  # Receive acknowlegments from workers
    
    while worker_rating_saved.qsize()>0 and time_out>0:
      new_rating = worker_rating_saved.get()  
      
      # Increase book_keeping count which shows number of updates pending in each block
      book_keeping[new_rating.block_index_i][new_rating.block_index_j] = book_keeping[new_rating.block_index_i][new_rating.block_index_j]+1
      time_out = time_out-1

      # if number of updates in a block is at least 10 and the block is unblocked then add it to set
      if book_keeping[new_rating.block_index_i][new_rating.block_index_j]>9 and is_blocked_row[new_rating.block_index_i]==False and is_blocked_column[new_rating.block_index_j]==False:
          set_s.add((new_rating.block_index_i , new_rating.block_index_j))

    

    while len(set_s)>0:  # while all blocks are not blocked, keeping assiging work to workers
    
        random_turn = random.choice(tuple(set_s))   # chosing a random unblocked  block from set  
        turn = Turn(random_turn[0], random_turn[1]) 
        worker_turn[random_turn[0]].put(turn)  # Assign it to worker
    
        set_s.remove(random_turn)  # remove it from set_s 

        # block row and column

        is_blocked_row[random_turn[0]]=True 
        is_blocked_column[random_turn[1]]=True    
        
        # reinitialize book_keeping to zero
        book_keeping[random_turn[0]][random_turn[1]]=0

        # Remove all blocks from set which lie on current row and column

        for i in range(len(book_keeping[random_turn[0]])):
          if book_keeping[random_turn[0]][i]>0:
            set_s.discard((random_turn[0],i))

        for i in range(len(book_keeping)):
          if book_keeping[i][random_turn[1]]>0:
            set_s.discard((i,random_turn[1]))

            

In [0]:
if __name__ == '__main__':

  d=5 # number of workers
  n=500 # number of users
  m=100 # number of items
  n_by_d = n//d  # number of users in each block
  m_by_d = m//d  # number of items in each block
  k=6  # number of latent features

In [0]:
  worker_turn_over =  multiprocessing.Queue()
  matrix_store_request = multiprocessing.Queue()
  matrix_store_data_update = multiprocessing.Queue()
  
  matrix_store_data_send = [ multiprocessing.Queue() for i in range(d) ]
  worker_turn =  [ multiprocessing.Queue() for i in range(d) ]
  worker_command = [ multiprocessing.Queue() for i in range(d) ]

  master_command = multiprocessing.Queue()
  matrix_store_command = multiprocessing.Queue()

  master_new_ratings = multiprocessing.Queue()
  worker_new_rating = [ multiprocessing.Queue() for i in range(d) ]
  worker_rating_saved = multiprocessing.Queue()


*   Creating all queues for communication bewteen all processes 

In [0]:
  rating_matrix = [[np.zeros((n_by_d,m_by_d)) for x in range(d)] for y in range(d)]
  is_updated = [[np.zeros((n_by_d,m_by_d)) for x in range(d)] for y in range(d)]

  is_blocked_row =  [False for i in range(d)] 
  is_blocked_column =  [False for i in range(d)] 
  
  matrix_U = [ U(k,n_by_d)  for i in range(d) ]
  matrix_V = [ V(k,m_by_d)  for i in range(d) ]

  book_keeping = [[0 for x in range(d)] for y in range(d)]

  master_is_blocked_row =  [False for i in range(d)] 
  master_is_blocked_column =  [False for i in range(d)] 

  set_s = set()

*  Creating and intializing different matrix, arrays and set required

In [0]:
  for i in range(n):
    items = random.randint(0,m//2)
    for j in range(items):
      j_cor = random.randint(0,m-1);
      rating_matrix[(i//n_by_d)][(j_cor//m_by_d)][i%n_by_d][j_cor%m_by_d]=random.randint(1,10)
  
  updates = []

  for i in range(d):
      for j in range(d):
          for k in range(n_by_d):
              for p in range(m_by_d):
                  if rating_matrix[i][j][k][p] != 0:
                      updates.append( Rating(k , p , rating_matrix[i][j][k][p] , i, j )) 
  
  random.shuffle(updates)
  
  rating_matrix = [[np.zeros((n_by_d,m_by_d)) for x in range(d)] for y in range(d)]

*   Assigning ratings to few items by each users 
*   Creagting matrix removes duplicates 
*   Creating an array which contains list of all ratings assignned
*   Randomly shuffle the list of updates to create a good stream data
*   Reinitialize the rating matrix to 0

In [0]:
  master_process = multiprocessing.Process(target=master, args=(set_s,  master_is_blocked_row , master_is_blocked_column  ,worker_rating_saved, book_keeping, master_new_ratings, worker_new_rating, worker_turn, worker_turn_over, d, master_command))
  
  worker_process =[ multiprocessing.Process(target=worker, args=(worker_rating_saved, is_updated[i], rating_matrix[i] , worker_new_rating[i] ,worker_turn[i], worker_turn_over, matrix_store_request , matrix_store_data_send[i] , matrix_store_data_update , worker_command[i])) for i in range(d) ]
  
  matrix_store_process = multiprocessing.Process(target=matrix_store, args=(rating_matrix , is_blocked_row,is_blocked_column,matrix_U, matrix_V , matrix_store_request , matrix_store_data_send , matrix_store_data_update, matrix_store_command))     


*  Creating all the processes and also passing required data, queues and matrices to them. 

In [0]:
  # Start all processes. We should start them according to topological ordering. 
  
  matrix_store_process.start()
  for i in range(d):
    worker_process[i].start() 
  master_process.start()

  print('Length of stream for demonstration purpose', len(updates),flush=True)

  # Keep sending data in form of stream to master
  for i in range(len(updates)):
      master_new_ratings.put(updates[i])

  # Wait for master to finish the work
  while worker_rating_saved.qsize()>0 or master_new_ratings.qsize()>0 or worker_turn_over.qsize()>0 :
      time.sleep(1)

  # Wait till workers and matrix store complete assigned work  
  time.sleep(5)

  # Print prediction and actual rating for a random block 
  # and also print mean square and mean absolute error 
  # to ensure if data is not getting overfitted for a particular block during update
  # Ask a random worker to do so 
  
  worker_command[0].put('Print')
  print('Main done',flush=True)

  # wait for 1 sec to ensure output gets printed by matrix store
  time.sleep(1)

  # End all the processes in reverse topological order

  master_command.put('Exit')
  for i in range(d):
    worker_command[i].put('Exit') 
  matrix_store_command.put('Exit')

Length of stream for demonstration purpose 10489
2   2  : Number of points received after last update :  24
2   2  Before MAE:  3.5681093390762633
2   2  Before MSE:  21.01604389245731
2   2  After MAE:  2.9796569843660854
2   2  After MSE:  15.361082757403992
2   0  : Number of points received after last update :  11
1   1  : Number of points received after last update :  39
0   3  : Number of points received after last update :  52
2   0  Before MAE:  4.215734029309944
3   4  : Number of points received after last update :  37
2   0  Before MSE:  28.59435882040866
1   1  Before MAE:  5.037494595622347
2   0  After MAE:  3.578857087904877
1   1  Before MSE:  27.064506738470413
3   4  Before MAE:  2.6177287191874514
1   1  After MAE:  4.157543780373169
1   1  After MSE:  18.46516761866685
2   0  After MSE:  20.940608754882227
3   4  Before MSE:  9.770430024784257
0   3  Before MAE:  3.7423602769035327
0   3  Before MSE:  19.8100618084479
3   4  After MAE:  2.081166601721616
0   3  Afte